In [1]:
from bs4 import BeautifulSoup
import requests






url = "https://www.ime.usp.br/bioinfo/pos/teses-dissertacoes"
page = requests.get(url)

soup = BeautifulSoup(page.text, "html.parser")

In [2]:
list_of_thesis = soup.find(class_ = "striped-table")

all_links = list_of_thesis.find_all("a")

In [3]:
list_of_pre_thesis_urls = []

for link in all_links:
    link_now = link.get('href')
    if "teses" in link_now:
        list_of_pre_thesis_urls.append(link_now)

In [4]:
list_of_pre_thesis_urls[0:5]

['http://teses.usp.br/autor.php?autor=57A9A0FBF285',
 'http://teses.usp.br/autor.php?autor=17A7188B7471',
 'http://teses.usp.br/autor.php?autor=17A86D22C7A1',
 'http://teses.usp.br/autor.php?autor=B7AAB42FA53B',
 'http://teses.usp.br/autor.php?autor=97A5A5761BB9']

In [5]:
def get_thesis_url(pre_thesis_url):
    page = requests.get(pre_thesis_url)
    soup = BeautifulSoup(page.text, "html.parser")
    box = soup.find(class_="dadosLinha dadosCor1")
    all_links = box.find_all("a")
    thesis_url = all_links[0].get("href")
    return(thesis_url)

def get_comittee_members (thesis_url):
    page = requests.get(thesis_url)
    soup = BeautifulSoup(page.text, "html.parser")
    commitee = soup.select("#CorpoTexto > div:nth-child(23)")[0].text
    commitee = commitee.split("\n\n")
    student = soup.select("#CorpoTexto > div:nth-child(9)")[0].text
    students_commitee_dict = dict()
    students_commitee_dict[student] = commitee
    return(students_commitee_dict)


In [ ]:
bioinfo_commitees_dict = dict()

for pre_thesis_url in list_of_pre_thesis_urls:
    try:
        thesis_url = get_thesis_url(pre_thesis_url)
        commitee_dict = get_comittee_members(thesis_url)
        bioinfo_commitees_dict.update(commitee_dict)
    except:
        None

In [7]:
import pandas as pd



## I do not recall how I've removed the PRESIDENTE label
bioinfo_commitees_df = pd.read_csv("bioinfo_commitees_df_dirty.csv")

bioinfo_commitees_df.columns = ['student', 'na', 'president', 'comitee', 'comitee', 'comitee', 'comitee']
bioinfo_commitees_df

,student,na,president,comitee,comitee,comitee,comitee
0,Junier Marrero Gutierrez,NaN,"Koide, Tie","Martinez, Cristina Elisa Alvarez","Pinheiro, Daniel Guariz","Silva Neto, José Freire da\n",NaN
1,Caio Rafael do Nascimento Santiago,NaN,"Digiampietri, Luciano Antonio","Chaim, Marcos Lordello","Ferreira, João Eduardo\n",NaN,NaN
2,Suzane de Andrade Barboza,NaN,"Digiampietri, Luciano Antonio","Amaral, Fábio Sarubbi Raposo do","Lima, Ariane Machado","Oliveira, Julio Cezar Franco de\n",NaN
3,Juan Enrique Faya Castillo,NaN,"Oliveira, Paulo Sérgio Lopes de","Benedetti, Celso Eduardo","Giuliatti, Silvana","Schechtman, Deborah\n",NaN
4,Felipe Prata Lima,NaN,"Setubal, João Carlos","Araujo, Welington Luiz de","Dias Neto, Emmanuel","Hashimoto, Ronaldo Fumio","Varani, Alessandro de Mello\n"
...,...,...,...,...,...,...,...
111,Gustavo Henrique Esteves,NaN,"Neves, Eduardo Jordao","Almeida, Sergio Verjovski de","Cesar Junior, Roberto Marcondes","Garcia, Nancy Lopes","Nunes, Luiz Roberto\n"
112,Bruno Monte de Castro,NaN,"Leonardi, Florencia Graciela","García, Jesús Enrique","Iambartsev, Anatoli\n",NaN,NaN
113,Cesar Armando Beltran Castañon,NaN,"Gruber, Arthur","Bruno, Odemir Martinez","Ferreira, Marcelo Urbano","Hirata, Nina Sumiko Tomita","Lotufo, Roberto de Alencar\n"
114,Bruno Sanchez de Araujo,NaN,"Machado-Lima, Ariane","Martins Junior, David Corrêa","Paraboni, Ivandre","Salvini, Rogerio Lopes\n",NaN


In [10]:




df = bioinfo_commitees_df.set_index('student')
df = df.stack()
df.index = df.index.rename('position', level=1)
df.name = "name"
df = df.reset_index()
df.head(10)

,student,position,name
0,Junier Marrero Gutierrez,president,"Koide, Tie"
1,Junier Marrero Gutierrez,comitee,"Martinez, Cristina Elisa Alvarez"
2,Junier Marrero Gutierrez,comitee,"Pinheiro, Daniel Guariz"
3,Junier Marrero Gutierrez,comitee,"Silva Neto, José Freire da\n"
4,Caio Rafael do Nascimento Santiago,president,"Digiampietri, Luciano Antonio"
5,Caio Rafael do Nascimento Santiago,comitee,"Chaim, Marcos Lordello"
6,Caio Rafael do Nascimento Santiago,comitee,"Ferreira, João Eduardo\n"
7,Suzane de Andrade Barboza,president,"Digiampietri, Luciano Antonio"
8,Suzane de Andrade Barboza,comitee,"Amaral, Fábio Sarubbi Raposo do"
9,Suzane de Andrade Barboza,comitee,"Lima, Ariane Machado"


In [38]:
def invert_names(name):
    name = name.replace("\n", "")
    try:
        bla = name.split(',')
        bla = bla[1] + " " + bla[0]
        return(bla)
    except:
        return(name)

professor_names = df['name']

df['name'] = professor_names.apply(invert_names)

In [40]:
tidy_bioinfo_comitees = df

In [42]:
tidy_bioinfo_comitees.to_csv("./tidy_bioinfo_comitees.csv")